# Retrieve and plot images and cutouts from the AL / JWST Server.

In [ ]:
# Example notebook to retrieve and plot images and cutouts from the AL/JWST Server via its API.
#   Utilizes the ALWS_utils.ipynb notebook, which contains API helper classes and methods.
#
#  Written by: Tom Hicks. 2/22/2022.
#  Last Modified: Updated for returned canonical results structure.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

%matplotlib inline

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.wcs import WCS
from astropy.utils.data import download_file
from astropy.visualization import MinMaxInterval, AsinhStretch, SqrtStretch, ImageNormalize
import astropy.units as u

#### Load additional "helper" code to help manage interaction with the API:

In [ ]:
%run ALWS_utils.ipynb

#### Create an instance of the API "helper" class which uses the default server base URL (both just loaded).

In [ ]:
api = AlwsApi()

#### Authorize an existing user, storing the resulting JWT access token in the instance of the class

In [ ]:
## Uncomment the following two lines to see the response when authorization fails:
# resp = api.authorize('jfake@nowhere.com', 'invalid_pwd')
# print(resp.json())
resp = api.authorize('jfake@nowhere.com', 'testydjango2')
# print(resp.json())

## Load and plot an entire image from the Astrolabe image server

#### First, let's see what images are available from the image server:

In [ ]:
resp = api.get('imgmd/')
[file.get('file_name') for file in get_results(resp)]

#### Since memory is limited, let's select a **small** image to display. (The following assumes that the `HorseHead.fits` file was listed above. If not, subsitute another filename below).

In [ ]:
imgURL = api.compose('img/fetch/?filename=HorseHead.fits')
image_file = download_file(imgURL, http_headers=api.auth_header)

In [ ]:
## Alternatively, one can download the bytes into memory and then read them directly:
# resp = api.get('img/fetch/?filename=HorseHead.fits')
# if (resp.status_code == 200):
#     image_file = io.BytesIO(resp.content)
# else:
#     print(resp.json())  # print error message    

In [ ]:
image_hdus = fits.open(image_file)

#### After downloading and opening the image, we can inspect its headers and properties.

In [ ]:
image_hdus.info()

In [ ]:
image = image_hdus[0].data
image.shape

#### Finally, we can plot the image.

In [ ]:
plt.imshow(image, cmap='magma')
plt.colorbar()

## Query for, load, and plot a cutout from the Astrolabe image server

#### First, get a cutout of 10 arc seconds from an image, with the specified filter, from the specified collection, which contains the specified point.

In [ ]:
params={'ra': '53.1617', 'dec': '-27.78', 'sizeArcSec': '10', 'filter': 'F356W', 'collection': 'DC20'}
resp = api.get('cuts/fetch_cutout/', params)
if (resp.status_code == 200):
    co_hdus = fits.open(io.BytesIO(resp.content))
else:
    print(resp.json())  # print error message

#### After downloading and opening the image, we can inspect its headers and properties.

In [ ]:
co_hdus.info()

In [ ]:
co_hdu = co_hdus[0]
wcs = WCS(co_hdu.header)
cutout = co_hdu.data
cutout.shape

#### Finally, we can plot the image data.

In [ ]:
# norm_cutout = ImageNormalize(cutout, interval=MinMaxInterval(), stretch=AsinhStretch())
norm_cutout = ImageNormalize(cutout, interval=MinMaxInterval(), stretch=SqrtStretch())

In [ ]:
fig = plt.figure(figsize=(14,14))
ax = fig.add_subplot(111, projection=wcs)
plt.imshow(cutout, origin='lower', cmap=plt.cm.gray, norm=norm_cutout)
plt.colorbar(pad=0.09, shrink=0.8)
ax.coords[0].set_axislabel('Right Ascension')
ax.coords[0].set_format_unit(u.degree, decimal=True)
ax.coords[1].set_axislabel('Declination')
ax.coords[1].set_format_unit(u.degree, decimal=True)

overlay = ax.get_coords_overlay('fk5')
overlay.grid(color='white', ls='dotted')
overlay[0].set_axislabel('RA')
overlay[1].set_axislabel(' ')